In [1]:
import csv
import math
import tsplib95
import statistics
import numpy as np
import pandas as pd
from itertools import groupby
from scipy.special import _logsumexp

### Normal functions of the algorithm

In [31]:
def herding(pop: list, Vt: list, fit: list, n: int, dimension: int, acc: list, time: list) -> list:
    
    fit1 = np.sort(fit,kind="heapsort") 
    idx = np.argsort(fit)
    pop1 = np.zeros((n, dimension))
    Vt1 = np.zeros((n, dimension))
    acc1 = np.zeros((n, dimension))
    time1 = np.zeros((n, dimension))

    for i in range(n):    
        pop1[i, :] = pop[idx[i], :]
        Vt1[i, :] = Vt[idx[i], :]
        acc1[i, :] = acc[i, :]
        time1[i] = time[i]
    
    return pop1, Vt1, fit1, acc1, time1

In [33]:
def update_v(Vt: list, n: int, dimension: int, acc: list, time: list, pop: list, fit: list, eye: int) -> list:
    #Choosing left and right dogs
    right_dog = np.random.randint(2,3)
    if (right_dog == 2):
        left_dog = 3
    else:
        left_dog = 2

    acc1 = np.ones((n,dimension))
    time1 = np.ones((n,1))
    Vt1 = np.copy(Vt)
    r = len(acc)
    l = len(acc)
    acc2 = np.zeros((r, l))
    fit1 = fit[l-1]

    #Finding Dg value to choose which sheep to gather and which to stalk
    fit2 = (fit[1] + fit[2]) / 2
    f = 0
    tempg = 0
    temps = 0

    #Setting parameters for eyeing
    if eye == 1:
        if fit[right_dog] < fit[left_dog]:
            acc2[left_dog, :] = -1 * acc[left_dog, :]
            f = left_dog
        else:
            acc2[right_dog, :] = -1 * acc[right_dog, :]
            f = right_dog

    for i in range(n):
        for j in range(dimension):
            #Velocity updation of dogs
            if (i <= 2):
                Vt1[i][j] = math.sqrt(np.power(Vt[i][j], 2) + (2 * acc[i][j]) * abs(pop[i][j]))
                
            #Velocity updation of sheep
            if (i > 2):
                if eye == 1:
                    Vt1[i][j] = math.sqrt(np.power(Vt1[f][j], 2) + (2*acc2[f][j]) * abs(pop[i][j]))

                else:
                    #Velocity updation of gathered sheep
                    if (fit1 - fit[i] > fit2 - fit[i]):
                        Vt1[i][j] = math.sqrt(np.power(Vt1[0,j], 2) + (2*acc[0,j]))
                        #tempg[i] = i        
                
                    #Velocity updation of stalked sheep
                    if (fit1 - fit[i] <= fit2 - fit[i]):
                        Vt1[i][j] = math.sqrt(np.power(Vt[right_dog][j]*math.tan(np.random.randint(1,89)), 2) + (2*acc[right_dog][j]*abs(pop[right_dog][j]))) + math.sqrt(np.power(Vt[left_dog][j]*math.tan(np.random.randint(91,179)), 2) + (2*acc[left_dog][j]*abs(pop[left_dog][j])))
                        Vt1[i][j] = (Vt1[i][j]) / 2
                        #temps[i] = i
    
    #Updation of time and acceleration
    for i in range(n):
        s = 0
        for j in range(dimension):
            acc1[i][j] = abs((Vt1[i][j]) - (Vt[i][j])) / (time[i][j])
            s += (_logsumexp.logsumexp(Vt1[i][j]) - _logsumexp.logsumexp(Vt[i][j])) / acc1[i][j]
            
        time1[i] = abs(np.mean(s))

    return Vt1, acc1, time1, right_dog, left_dog, tempg, temps

In [4]:
def check(pop: list, n: int, dimension: int, acc: list, Vt: list, time: list) -> list:
    pop1 = np.copy(pop)
    acc1 = np.copy(acc)
    time1 = np.copy(time)
    Vt1 = np.copy(Vt)
    for i in range(n):
        for j in range(dimension):

            if (pop[i][j] >= 1 or pop[i][j] <= 0 or pop[i][j] == 0):
                pop1[i][j] = np.random.random() * 1
                acc1[i][j] = np.random.random()
                time1[i] = np.random.random()
    
    for i in range(n):
        for j in range(dimension):
            if (math.isnan(acc[i][j]) == 1 or acc[i][j] == 0):
                pop1[i][j] = np.random.random() * 1
                acc1[i][j] = np.random.random()
                time1[i] = np.random.random()

    for i in range(n):
        for j in range(dimension):
            if (math.isnan(Vt[i][j]) == 1 or Vt[i][j] == 0):
                pop1[i][j] = np.random.random() * 1
                acc1[i][j] = np.random.random()
                time1[i] = np.random.random()

    for i in range(n):
        for j in range(dimension):
            if (math.isnan(time1[i]) == 1 or time1[i] == 0):
                pop1[i][j] = np.random.random() * 1
                acc1[i][j] = np.random.random()
                time1[i] = np.random.random()


    return pop1, acc1, time1, Vt1

### Discretization of the problem

In [39]:
def discrete_fitness(pop, n, objf):
    # How will discretization work for the fitness function?
    # Change this function to made the discretization.
    fit1 = np.zeros((n, 1))

    for i in range(len(pop)):
        for j in range(len(objf)):
            if pop[i][j] == 1:
                fit1[i] = objf[i][j]
    
    #best fit and position
    best_fit = np.min(fit1)
    pos = np.argmin(fit1)
    
    return fit1, best_fit, pos 

In [6]:
def discrete_generate(n: int, dimension: int) -> list:
    boundary_no = 1
    pop = np.zeros((n, dimension))

    if boundary_no == 1:
        pop = np.random.randint(0, 1+1, size=(n,dimension))

    x = np.random.random(size = (n, dimension))
    
    return pop, x

In [7]:
def discretizing(value, max_value):
    if (value >= (max_value/2)):
        value = 1
    else:
        value = 0
    
    return value

In [8]:
# No needed of modifying, keep the original function
def discrete_update(Vt: list, time: list, acc: list, n: int, dimension: int, eye: int) -> list:
    pop1 = np.zeros((n,dimension))
    for i in range(n):
        for j in range(dimension):
            #Updating the position of dogs
            if (i <= 2):
                pop1[i][j] = discretizing(Vt[i][j], np.nanmax(Vt[i])) * discretizing(time[i], np.nanmax(time)) + (1/2) * discretizing(acc[i][j], np.nanmax(acc[i])) * (np.power(discretizing(time[i], np.nanmax(time)), 2))
                
            #Updating position of sheep
            if (i > 2):
                if eye == 1:
                    pop1[i][j] = discretizing(Vt[i][j], np.nanmax(Vt[i])) * discretizing(time[i], np.nanmax(time)) - (1/2) * discretizing(acc[i][j], np.nanmax(acc[i])) * (np.power(discretizing(time[i], np.nanmax(time)), 2))
                
                else:
                    pop1[i][j] = discretizing(Vt[i][j], np.nanmax(Vt[i])) * discretizing(time[i], np.nanmax(time)) + (1/2) * discretizing(acc[i][j], np.nanmax(acc[i])) * (np.power(discretizing(time[i], np.nanmax(time)), 2))
                
    return pop1

Idea: Verify the route every loop just with the cities and after 
the iteration, call the punishment function.

Idea 2: Always verify if the route is valid.}

### Funtions to define  the route

In [13]:
# The first city is 0
# Need to put at least one sheep per city
# Create a function to make the reward strategy

# Create a random number each iteration and after that made a line of the matrix receive that number
# Check ✔️
def generate_city(n: int, dim: int) -> list:
    city_matrix = np.zeros((n, dim))
    for i in range(n):
        sheep = np.random.randint(0, dim)
        for j in range(dim):
            if (i != j):
                if (sheep == j):
                    city_matrix[i][j] = 1
    
    x = np.random.random(size = (n, dim))
    
    return city_matrix, x

# Debug function
# There's no need to use this function in the main algorithm, just in the test file
def print_position_of_sheep(city_matrix: list):
    for i in range(len(city_matrix)):
        for j in range(len(city_matrix)):
            if (city_matrix[i][j] != 0):
                print("Position -> ",j,"\n")

# Function to verify the route
# Verify if the list route has different values
# Check
def verify_route(city_matrix: list) -> bool:
    # Append each position in a list, after that verify if the route is valid in a different ways
    valid = True
    route = []
    route.append(0)
    for i in range(len(city_matrix)):
        for j in range(len(city_matrix)):
            if (city_matrix[i][j] == 1):
                route.append(j)
                if (i == (len(city_matrix) - 1) and j != 0):
                    valid = False
    
    print("Route -> ", route,"\n")

    return route, valid

# Check 
def same_cities(route: list) -> bool:
    same = False
    aux = route[-1]
    i = 1
    j = 1
    while i != (len(route) - 2):
        if (aux == route[j]):
            same = True
            #print("Equal value -> ", route[j], "\nAux value -> ", aux,"\n")
            break
        j += 1
        if (j == (len(route) - 1)):
            aux = route[i]
            i += 1
            j = 1

    return same    

# Check 
def punishment(city_matrix: list, weight: int):
    route, aux_verify = verify_route(city_matrix)
    aux_route = same_cities(route)
    position = route[1]
    if (aux_route == True or aux_verify == False):
        city_matrix[0][position] *= np.power(weight, 2)
    
    return city_matrix

### Application of TSP

In [10]:
problem = tsplib95.load('../data/domain/bays29.tsp')

In [38]:
#Population size
n = problem.dimension

#Maximum no. of iterations
gen = 200

#Optimization function name
dim = problem.dimension

#Intialize the population(init_p-Population,acc-acceleration of each individual)
init_p, acc = generate_city(n, dim)

#Vt = velocity of each individuals
Vt = np.zeros((n, dim))

#Time of each individual
time = np.random.random(size = (n,1))

#Max fitness value/
fopt = math.inf
#Variable to store fitness
fit = np.zeros((n, 1))
pop = init_p
#k = counter variable for iterations required for Eyeing mechanism
k = 1
fopt_1 = np.zeros((gen, 1))
# Generate a route and at the same time verify if this route is valid
# Needs to generate before the loop and adjust in the iterations till turns it valid
route, valid = verify_route(pop)

for g in range(gen):
    #Calculate fitness of indivuals
    fit, maxf, pos = discrete_fitness(pop, n, getattr(problem, 'edge_weights'))
    #print("Fit -> ",fit,"\n")
    maxf = 1
    eye = 0
    if g == 0:
        fopt = maxf
    
    #Finding the optimum fitness value
    if fopt > maxf:
        fopt = maxf
    
    fopt_1[g] = fopt
    if g > 0:
        if fopt_1[g] > fopt_1[g-1]:
            k += 1
            if k > 5:
                eye = 1
                k = 0
    
    pop, Vt, fit, acc, time = herding(pop, Vt, fit, n, dim, acc, time)

    Vt, acc, time, r1, l1, tempg, temps = update_v(Vt, n, dim, acc, time, pop, fit, eye)

    pop = discrete_update(Vt, time, acc, n, dim, eye)

    pop, acc, time, Vt = check(pop, n, dim, acc, Vt, time)
    

Route? ->  5 

Route? ->  0 



C:\Users\artur\AppData\Local\Temp/ipykernel_3388/2702834189.py:60: RuntimeWarning: invalid value encountered in double_scalars
  s += (_logsumexp.logsumexp(Vt1[i][j]) - _logsumexp.logsumexp(Vt[i][j])) / acc1[i][j]


Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 

Route? ->  0 



KeyboardInterrupt: 